In [13]:
import time
from src.mitnanex import run
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from Bio import SeqIO
from sourmash import MinHash
import khmer

In [2]:

## FASTA file is lighter
# convert_fq_to_fa(
#     "test/aedes_vexans_all_reads_subsample_sorted_length.fastq",
#     "test/aedes_vexans_all_reads_subsample_sorted_length.fasta",
# )
records_dict = SeqIO.to_dict(
        SeqIO.parse("test/aedes_vexans_all_reads_subsample_sorted_length.fasta", "fasta")
    )
a = run()

# I need to plot the coverage of clusters
coverages_df = pd.DataFrame(
    {
        "coverage": [i.coverage for i in a.clusters],
        "longest_read_len": [i.longest_read_length for i in a.clusters],
        "id_longest_read": [i.longest_read_id for i in a.clusters],
        "id_cluster":[i.id_cluster for i in a.clusters],
    }
)



In [3]:
# Test on the first 30 clusters
prueba = coverages_df.sort_values(by='coverage', ascending=False).head(30)
for id in prueba['id_longest_read']:
    records_dict[id].seq

# Oligo composition

Once I've detected the clusters with higher coverage, which I expect includes mitochondria and contamination, whether nuclear or external, I have to purify these groups. To do this I will use the oligo composition as it is used during metagenomics binning.

I will create a script to get the kmers. The kmer size will be 3 to solve two things: 1. Intrinsec error from Nanopore, 2 smaller set as possible (4**3 possible kmers). 

ModuleNotFoundError: No module named 'jellyfish'